In [2]:
import numpy as np
import scipy.sparse
import sympy
import sklearn.datasets
import sklearn.feature_extraction.text
import umap
#import umap.plot
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
primes = list(sympy.primerange(2, 110000))
prime_to_column = {p:i for i, p in enumerate(primes)}


In [4]:
%%time
lil_matrix_rows = []
lil_matrix_data = []
for n in range(100000):
    prime_factors = sympy.primefactors(n)
    lil_matrix_rows.append([prime_to_column[p] for p in prime_factors])
    lil_matrix_data.append([1] * len(prime_factors))


CPU times: user 1.06 s, sys: 63 ms, total: 1.12 s
Wall time: 1.18 s


In [5]:
factor_matrix = scipy.sparse.lil_matrix((len(lil_matrix_rows), len(primes)), dtype=np.float32)
factor_matrix.rows = np.array(lil_matrix_rows)
factor_matrix.data = np.array(lil_matrix_data)
factor_matrix

/var/folders/z0/b8gp2l4j50l0rgwy21ps2ltm0000gn/T/ipykernel_15664/3118264754.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  factor_matrix.rows = np.array(lil_matrix_rows)
/var/folders/z0/b8gp2l4j50l0rgwy21ps2ltm0000gn/T/ipykernel_15664/3118264754.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  factor_matrix.data = np.array(lil_matrix_data)


<100000x10453 sparse matrix of type '<class 'numpy.float32'>'
	with 266398 stored elements in List of Lists format>

In [6]:
factor_matrix

<100000x10453 sparse matrix of type '<class 'numpy.float32'>'
	with 266398 stored elements in List of Lists format>

In [8]:
mapper = umap.UMAP(metric='cosine', random_state=42, low_memory=True).fit(factor_matrix)


AttributeError: module 'umap' has no attribute 'UMAP'

In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from umap import UMAP

# Make a toy dataset
X, y = make_classification(
    n_samples=1000,
    n_features=300,
    n_informative=250,
    n_redundant=0,
    n_repeated=0,
    n_classes=2,
    random_state=1212,
)

# Split the dataset into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Classification with a linear SVM
svc = LinearSVC(dual=False, random_state=123)
params_grid = {"C": [10 ** k for k in range(-3, 4)]}
clf = GridSearchCV(svc, params_grid)
clf.fit(X_train, y_train)
print(
    "Accuracy on the test set with raw data: {:.3f}".format(clf.score(X_test, y_test))
)

# Transformation with UMAP followed by classification with a linear SVM
umap = UMAP(random_state=456)
pipeline = Pipeline([("umap", umap), ("svc", svc)])
params_grid_pipeline = {
    "umap__n_neighbors": [5, 20],
    "umap__n_components": [15, 25, 50],
    "svc__C": [10 ** k for k in range(-3, 4)],
}


clf_pipeline = GridSearchCV(pipeline, params_grid_pipeline)
clf_pipeline.fit(X_train, y_train)
print(
    "Accuracy on the test set with UMAP transformation: {:.3f}".format(
        clf_pipeline.score(X_test, y_test)
    )
)

Accuracy on the test set with raw data: 0.755


OMP: Info #270: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Accuracy on the test set with UMAP transformation: 0.935


In [6]:
umap = UMAP(random_state=456)

In [ ]:
!pip3 uninstall umap

Found existing installation: umap 0.1.1
Uninstalling umap-0.1.1:
  Would remove:
    /Users/michaelallwright/opt/anaconda3/lib/python3.9/site-packages/umap-0.1.1.dist-info/*
    /Users/michaelallwright/opt/anaconda3/lib/python3.9/site-packages/umap/*
Proceed (Y/n)? 